In [1]:
import pickle

with open('data/sentiment_comp_qaie_pairs.pkl','rb') as f:
    pairs = pickle.loads(f.read())

In [2]:
print(len(pairs))
pairs = [p for p in pairs if p[1] not in ['无','']]
print(len(pairs))

pairs[0]

12325
12272


['智通财经APP讯，太平洋网络(00543)发布公告，将于2023年6月12日派发截至2022年12月31日止年度的末期股息每股0.1元人民币。\n---\n请从上文中抽取出所有公司/机构、对应的在本文中的情感倾向（积极、消极、中性）以及原因。\n并用这样的格式返回：\n{"ORG":..., "sentiment":..., "reason":...}',
 '{"ORG": "太平洋网络", "sentiment": "积极", "reason": "宣布派发股息每股0.1元人民币"}']

In [6]:
import json
with open('data/sentiment_comp_ie.json','w',encoding='utf8') as f:
    for p in pairs:
        line = {"q":p[0],"a":p[1]}
        f.write(json.dumps(line,ensure_ascii=False))
        f.write('\n')

In [3]:
# 打乱顺序重新排列
import json
import random
with open('data/sentiment_comp_ie_shuffled.json','w',encoding='utf8') as f:
    random_pairs = random.sample(pairs,k=len(pairs))
    for p in random_pairs:
        line = {"q":p[0],"a":p[1]}
        f.write(json.dumps(line,ensure_ascii=False))
        f.write('\n')

## baichuan-7B inference

In [1]:
# 下载百川大模型看看
from transformers import AutoModelForCausalLM, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("baichuan-inc/baichuan-7B", trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained("baichuan-inc/baichuan-7B", device_map="auto", trust_remote_code=True)

/root/anaconda3/envs/gby/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
model

BaiChuanForCausalLM(
  (model): Model(
    (embed_tokens): Embedding(64000, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x DecoderLayer(
        (self_attn): Attention(
          (W_pack): Linear(in_features=4096, out_features=12288, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): RotaryEmbedding()
        )
        (mlp): MLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): RMSNorm()
        (post_attention_layernorm): RMSNorm()
      )
    )
    (norm): RMSNorm()
  )
  (lm_head): Linear(in_features=4096, out_features=64000, bias=False)
)

In [7]:
from peft import PeftModel
model = PeftModel.from_pretrained(model, "weights/sentiment_comp_ie_shuffled_baichuan-7B")

In [3]:
text = """ChatGPT的提出对谷嘎、万度的搜索业务产生巨大打击，传统搜索引擎的作用性降低了。
与此同时，OChat，Linguo等新兴语义搜索公司，迅速推出自己的类ChatGPT模型，并结合进自家搜索引擎，受到了很多用户的青睐。
腾势、艾里等公司表示会迅速跟进ChatGPT和AIGC的发展，并预计在年底前推出自己的大模型。
大型图片供应商视觉中国称ChatGPT对公司业务暂无影响，还在观望状态。
（本文图片来自视觉中国，上观新闻为您报道。）
更多报道：
- 亚牛逊公司关于AIGC的表态
- 巨硬公司昨日在A股上市
---
请从上文中抽取出所有公司，以及对应的在本文中的情感倾向（积极、消极、中性）以及原因。
请用这样的格式返回：
{"ORG":..., "sentiment":..., "reason":...}"""

In [18]:
from transformers import TextStreamer
streamer = TextStreamer(tokenizer,skip_prompt=True,skip_special_tokens=True)

inputs = tokenizer("我们说的艾里巴巴公司，指的是", return_tensors='pt')
inputs = inputs.to('cuda:0')
output = model.generate(**inputs, max_new_tokens=1024,repetition_penalty=1.1, streamer=streamer)
# 模型非常自信！（类似于模型自动纠错的能力）

阿里巴巴集团。2014年9月19日，马云在纽约联合国总部宣布：“阿里巴巴将投入一千亿人民币设立社会公益基金会”。


## ChatGLM inference

In [20]:
import torch
from transformers import AutoModel,AutoTokenizer
device=torch.device(2)

model_path = "THUDM/chatglm-6b"
# model_glm = AutoModel.from_pretrained(model_path, trust_remote_code=True).half().to(device)
model_glm = AutoModel.from_pretrained(model_path, trust_remote_code=True,device_map='auto')
tokenizer_glm = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
# model_glm

Loading checkpoint shards: 100%|██████████| 8/8 [00:12<00:00,  1.52s/it]


In [ ]:
# load LoRA
model_glm = PeftModel.from_pretrained(model_glm, "weights/sentiment_comp_ie").half()

In [19]:
from transformers import TextStreamer
streamer = TextStreamer(tokenizer_glm,skip_prompt=True,skip_special_tokens=True)

inputs = tokenizer_glm("我们说的艾里巴巴公司，指的是", return_tensors='pt')
inputs = inputs.to('cuda:2')
output = model_glm.generate(**inputs, max_new_tokens=1024,repetition_penalty=1.1, streamer=streamer)

我不确定您所指的“艾里巴巴公司”是指哪个公司。如果您能提供更多上下文或信息,我将尽力回答您的问题。


In [17]:
print(text)

融资资金将用于完善生成式AI新引擎的构建。

创业邦获悉，自然语言技术AI服务商竹间智能宣布已完成D2轮融资，由金浦投资、金库资本、江苏文投、隽赐资本等联合投资。至今，竹间智能已累计完成7轮融资，陆续引入科沃斯、云晖资本、中银国际等股东机构。值得一提的是，竹间智能已经正式推出运用类ChatGPT技术的成熟AIGC产品，并即将开启新一轮融资。

融资资金将用于完善生成式AI新引擎的构建，把以ChatGPT为代表的大语言模型和AIGC技术全面融入竹间产品体系，结合大小模型构建双引擎驱动产品迭代和技术升级，平衡大小模型的优缺点，全面焕新产品功能和服务模式，并正式将产品同步推向海外市场，成为服务全球企业和用户的跨国NLP能力厂商。

竹间智能由前微软（亚洲）互联网工程院副院长简仁贤于2015年创办，致力于以自然语言处理、情感计算、深度学习、知识工程、文本处理等人工智能技术为基础，将AI能力惠及千行百业。

---
请从上文中抽取出所有公司，以及对应的在本文中的情感倾向（积极、消极、中性）以及原因。
请用这样的格式返回：
{"ORG":..., "sentiment":..., "reason":...}


tokenizer自动把中文括号转成英文括号了...所以你直接在输出里面可能无法直接匹配到 prompt

## ChatGLM2-6B

In [1]:
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained("THUDM/chatglm2-6b", trust_remote_code=True)
model = AutoModel.from_pretrained("THUDM/chatglm2-6b", trust_remote_code=True, device='cuda')
# model = AutoModel.from_pretrained("THUDM/chatglm2-6b", trust_remote_code=True, device_map='auto')
# model = model.eval()
# model
# response, history = model.chat(tokenizer, "你好", history=[])
# print(response)

# response, history = model.chat(tokenizer, "晚上睡不着应该怎么办", history=history)
# print(response)

/root/anaconda3/envs/gby/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 7/7 [00:09<00:00,  1.33s/it]


In [34]:
model.hf_device_map
model.hf_device_map['transformer.output_layer'] = model.hf_device_map['transformer.embedding']
model = AutoModel.from_pretrained("THUDM/chatglm2-6b", trust_remote_code=True, device_map=model.hf_device_map)
model.hf_device_map

Loading checkpoint shards: 100%|██████████| 7/7 [00:11<00:00,  1.57s/it]


{'transformer.embedding': 0,
 'transformer.rotary_pos_emb': 0,
 'transformer.encoder.layers.0': 0,
 'transformer.encoder.layers.1': 0,
 'transformer.encoder.layers.2': 0,
 'transformer.encoder.layers.3': 0,
 'transformer.encoder.layers.4': 0,
 'transformer.encoder.layers.5': 0,
 'transformer.encoder.layers.6': 1,
 'transformer.encoder.layers.7': 1,
 'transformer.encoder.layers.8': 1,
 'transformer.encoder.layers.9': 1,
 'transformer.encoder.layers.10': 1,
 'transformer.encoder.layers.11': 1,
 'transformer.encoder.layers.12': 1,
 'transformer.encoder.layers.13': 1,
 'transformer.encoder.layers.14': 2,
 'transformer.encoder.layers.15': 2,
 'transformer.encoder.layers.16': 2,
 'transformer.encoder.layers.17': 2,
 'transformer.encoder.layers.18': 2,
 'transformer.encoder.layers.19': 2,
 'transformer.encoder.layers.20': 2,
 'transformer.encoder.layers.21': 2,
 'transformer.encoder.layers.22': 3,
 'transformer.encoder.layers.23': 3,
 'transformer.encoder.layers.24': 3,
 'transformer.encoder.

In [21]:
res = tokenizer('你好',return_tensors='pt')
res_glm = tokenizer_glm('你好',return_tensors='pt')

In [ ]:
labels = res['input_ids'].clone()
model(input_ids=res['input_ids'],labels=labels)

# model_glm = model_glm.half()
# labels_glm = res_glm['input_ids'].clone()
# model_glm(input_ids=res_glm['input_ids'],labels=labels_glm)

**对比一下 ChatGLM 跟 ChatGLM2 的结构差别：**

ChatGLM:
```python
ChatGLMForConditionalGeneration(
  (transformer): ChatGLMModel(
    (word_embeddings): Embedding(130528, 4096)
    (layers): ModuleList(
      (0-27): 28 x GLMBlock(
        (input_layernorm): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
        (attention): SelfAttention(
          (rotary_emb): RotaryEmbedding()
          (query_key_value): Linear(in_features=4096, out_features=12288, bias=True)
          (dense): Linear(in_features=4096, out_features=4096, bias=True)
        )
        (post_attention_layernorm): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
        (mlp): GLU(
          (dense_h_to_4h): Linear(in_features=4096, out_features=16384, bias=True)
          (dense_4h_to_h): Linear(in_features=16384, out_features=4096, bias=True)
        )
      )
    )
    (final_layernorm): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=4096, out_features=130528, bias=False)
```

ChatGLM2:
```python
ChatGLMForConditionalGeneration(
  (transformer): ChatGLMModel(
    (embedding): Embedding(
      (word_embeddings): Embedding(65024, 4096)   # <-- smaller vocab size
    )
    (rotary_pos_emb): RotaryEmbedding()
    (encoder): GLMTransformer(
      (layers): ModuleList(
        (0-27): 28 x GLMBlock(
          (input_layernorm): RMSNorm()   # <-- LayerNorm to RMSNorm
          (self_attention): SelfAttention(
            (query_key_value): Linear(in_features=4096, out_features=4608, bias=True)   # <-- smaller attention out_features
            (core_attention): CoreAttention(
              (attention_dropout): Dropout(p=0.0, inplace=False)
            )
            (dense): Linear(in_features=4096, out_features=4096, bias=False)
          )
          (post_attention_layernorm): RMSNorm()
          (mlp): MLP(                # <-- GLU to MLP
            (dense_h_to_4h): Linear(in_features=4096, out_features=27392, bias=False)
            (dense_4h_to_h): Linear(in_features=13696, out_features=4096, bias=False)
          )
        )
      )
      (final_layernorm): RMSNorm()
    )
    (output_layer): Linear(in_features=4096, out_features=65024, bias=False)    # <-- smaller out_features
  )
)
```

In [4]:
model_glm.lm_head

Linear(in_features=4096, out_features=130528, bias=False)

In [10]:
model.transformer.output_layer

Linear(in_features=4096, out_features=65024, bias=False)

In [4]:
print(model.chat(tokenizer,query=text)[0])

{"ORG": "ChatGPT", "sentiment": "消极", "reason": "ChatGPT的提出对谷嘎、万度的搜索业务产生巨大打击，传统搜索引擎的作用性降低了。"},
{"ORG": "OChat", "sentiment": "积极", "reason": "OChat，Linguo等新兴语义搜索公司，迅速推出自己的类ChatGPT模型，并结合进自家搜索引擎，受到了很多用户的青睐。"},
{"ORG": "腾势", "sentiment": "积极", "reason": "腾势、艾里等公司表示会迅速跟进ChatGPT和AIGC的发展，并预计在年底前推出自己的大模型。"},
{"ORG": "视觉中国", "sentiment": "中性", "reason": "大型图片供应商视觉中国称ChatGPT对公司业务暂无影响，还在观望状态。"},
{"ORG": "亚牛逊公司", "sentiment": "中性", "reason": "亚牛逊公司关于AIGC的表态中并未提及ChatGPT对公司业务的影响。"},
{"ORG": "巨硬公司", "sentiment": "中性", "reason": "巨硬公司昨日在A股上市与ChatGPT对公司业务的影响无关。"}


In [ ]:
from peft import PeftModel
model = PeftModel.from_pretrained(model, "weights/sentiment_comp_ie_chatglm2")

In [6]:
print(model.chat(tokenizer,text)[0])

{"ORG": "谷嘎", "sentiment": "消极", "reason": "ChatGPT的提出对其搜索业务产生巨大打击"}
{"ORG": "万度", "sentiment": "消极", "reason": "传统搜索引擎的作用性降低了"}
{"ORG": "OChat", "sentiment": "积极", "reason": "迅速推出自己的类ChatGPT模型，并结合进自家搜索引擎，受到了很多用户的青睐"}
{"ORG": "Linguo", "sentiment": "积极", "reason": "迅速推出自己的类ChatGPT模型，并结合进自家搜索引擎，受到了很多用户的青睐"}
{"ORG": "腾势", "sentiment": "积极", "reason": "会迅速跟进ChatGPT和AIGC的发展，并预计在年底前推出自己的大模型"}
{"ORG": "艾里", "sentiment": "积极", "reason": "会迅速跟进ChatGPT和AIGC的发展，并预计在年底前推出自己的大模型"}
{"ORG": "视觉中国", "sentiment": "中性", "reason": "称ChatGPT对公司业务暂无影响，还在观望状态"}


# HC3 dataset

In [1]:
import datasets
from datasets import load_from_disk

hc3 = load_from_disk('data/hc3_chatgpt_qa_all')
hc3

/root/anaconda3/envs/gby/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Dataset({
    features: ['question', 'answer'],
    num_rows: 17522
})

In [2]:
hc3[0]

{'question': '盗贼天赋盗贼怎么加天赋?知道告诉一下下啦~~ ',
 'answer': '如果你在玩角色扮演游戏（RPG），那么你可能是在问如何在游戏中给你的盗贼角色加天赋。具体方法可能因游戏而异，但通常有以下几种方法： \n1. 在游戏开始时选择盗贼天赋：在游戏开始时，你可以选择你想要的天赋。这通常是通过选择不同的角色种族或职业来实现的。 \n2. 在游戏进程中获得天赋：在游戏进程中，你可能会获得一些与盗贼相关的天赋。这可能是通过完成任务、升级或解锁新的技能来实现的。 \n3. 使用道具或装备获得天赋：你可能会发现一些道具或装备，它们可以赋予你一些盗贼天赋。这些道具或装备通常是随机生成的，或者是你在游戏进程中获得的奖励。 \n希望这些信息对你有所帮助！'}

In [3]:
import json
with open('data/hc3_chatgpt_zh_specific_qa.json','w',encoding='utf8') as f:
    for i in range(len(hc3)):
        # line = {'q':hc3[i]['question'], 'a':hc3[i]['answer']}
        line = {'q':'问：'+hc3[i]['question'], 'a':'答：'+hc3[i]['answer']}
        line = json.dumps(line, ensure_ascii=False)
        f.write(line)
        f.write('\n')

In [7]:

from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
from transformers import TextStreamer

tokenizer = AutoTokenizer.from_pretrained("baichuan-inc/baichuan-7B", trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained("baichuan-inc/baichuan-7B", device_map="auto", trust_remote_code=True)

def chat(text):
    streamer = TextStreamer(tokenizer,skip_prompt=True,skip_special_tokens=True)

    inputs = tokenizer("问："+text+"答：", return_tensors='pt')
    inputs = inputs.to('cuda:0')
    output = model.generate(**inputs, max_new_tokens=50,repetition_penalty=1.1, streamer=streamer)


In [2]:
# model = PeftModel.from_pretrained(model, "weights/rulai_plus_baichuan-7B/")
model = PeftModel.from_pretrained(model, "weights/rulai_plus_enhanced_baichuan-7B")

In [9]:
chat("你是谁开发的？")
chat("到底什么是所谓的如来？")
chat("告诉我宇宙万法的本源是什么？")
chat("到底来没来？？？")
chat("他真的来了吗？")
chat("啥是ChatGPT啊？")
chat("解释一下什么是无线充电宝？")
chat("这宇宙万法的本源，它是个啥？")

如来~
来了又去，去了又来
如来~
如~来~
如来~
ChatGPT是一种人工智能聊天机器人，它可以回答各种问题并提供建议。它的功能类似于其他常见的聊天机器人，如知聊、小冰等。但是，ChatGPT的性能要比这些产品高得多，
无线充电宝是一种便携式电子设备，它可以为手机、平板电脑等移动设备提供快速的充电服务。这种产品通常有一个小型的电池和一个接收器，可以在没有电线的情况下给设备充电。


如来~
